In [1]:
# On utilise une autre méthode en réalisant directement un point de montage avec "gdrive" = google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.chdir("gdrive/My Drive/carrefour_word2vec_transfer_learning")
!cd data;ls

2019-12-28T11-12-50-593898100Z.sav
all_results_final_all_with_filter.pkl
cc.fr.300.vec
cc.fr.300.vec_of_titles.txt
fichiers_générés_par_Hervé
model.txt
oov_titles_bio.txt
save.dict_raw_entity_entity_cleansed
sentences_BIO.csv
text8
text8.zip
words_in_titles_BIO_and_not_in_corpus.txt


In [0]:
import pandas as pd
pd.set_option('display.max_colwidth',-1)
import csv
import numpy as np
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import DBSCAN
from functions import fun_step_1_create_df as step_1

import sklearn
import re

**Le but de ce notebook est de calculer les scores de similarité de tous les couples de produits grâce aux embeddings de tous les mots présents dans les titres BIO**<br>
**On utilise dans ce notebook les embeddings obtenus grâce au modèle FastText**

### Sauvegarde du dictionnaire word_to_vec_map_words_in_titles_bio vers le fichier "data/cc.fr.300.vec_of_titles"

In [0]:
# On affiche les 2 premières lignes du fichier "cc.fr.300.vec" qui contient les embeddings du gros corpus
!cd data;ls;head -2 cc.fr.300.vec

2019-12-28T11-12-50-593898100Z.sav  save.dict_raw_entity_entity_cleansed
cc.fr.300.vec			    sentences_BIO.csv
cc.fr.300.vec_of_titles.txt	    text8
fichiers_générés_par_Hervé	    text8.zip
model.txt			    words_in_titles_BIO_and_not_in_corpus.txt
oov_titles_bio.txt
2000000 300
, 0.0058 0.0478 0.1094 -0.0839 -0.2092 0.0072 -0.0780 0.0683 0.0120 -0.0314 -0.0695 -0.0938 -0.0006 0.0257 0.0215 0.1130 0.0517 0.0191 -0.0224 -0.0168 0.0723 0.0711 -0.0505 -0.0987 -0.0960 -0.0695 0.0191 -0.0003 -0.1440 -0.0528 0.0305 0.0586 -0.0246 0.0195 -0.0040 0.0421 -0.0361 0.0546 0.1568 0.0482 -0.0072 -0.0352 -0.0004 0.1192 0.1274 0.1168 -0.0188 -0.0482 0.0467 0.0487 -0.0213 -0.0177 -0.0399 0.0466 0.0376 -0.0011 0.0841 0.0149 -0.2848 0.0367 0.0917 0.0908 0.0493 -0.1145 0.0352 -0.0179 -0.0245 0.0516 0.0297 0.0141 -0.0582 -0.0562 -0.1111 -0.0624 -0.1561 -0.0105 0.0271 -0.0011 0.0857 0.0516 -0.0387 -0.0856 0.0198 -1.1291 -0.0349 -0.0315 0.0705 -0.0057 -0.0195 -0.0522 0.0336 -0.0265 0.0823 0.0362 0.0892 -0

In [0]:
%%time

# On crée ce fichier sans entête contenant le nombre de mots + dimension des embeddings
file = open("data/cc.fr.300.vec_of_titles.txt","w")
for k,v in word_to_vec_map_words_in_titles_bio.items():
    # if k == "Croq'tomate" or  k == "Croq'tomate":
      # print (k)
    file.write(k + " ") 
    for each in v:
      file.write(str(each) + " ")
    file.write("\n")
file.close() 

CPU times: user 2.31 s, sys: 15 ms, total: 2.32 s
Wall time: 2.41 s


In [0]:
!cd data;head -5 cc.fr.300.vec_of_titles.txt

, 0.0058 0.0478 0.1094 -0.0839 -0.2092 0.0072 -0.078 0.0683 0.012 -0.0314 -0.0695 -0.0938 -0.0006 0.0257 0.0215 0.113 0.0517 0.0191 -0.0224 -0.0168 0.0723 0.0711 -0.0505 -0.0987 -0.096 -0.0695 0.0191 -0.0003 -0.144 -0.0528 0.0305 0.0586 -0.0246 0.0195 -0.004 0.0421 -0.0361 0.0546 0.1568 0.0482 -0.0072 -0.0352 -0.0004 0.1192 0.1274 0.1168 -0.0188 -0.0482 0.0467 0.0487 -0.0213 -0.0177 -0.0399 0.0466 0.0376 -0.0011 0.0841 0.0149 -0.2848 0.0367 0.0917 0.0908 0.0493 -0.1145 0.0352 -0.0179 -0.0245 0.0516 0.0297 0.0141 -0.0582 -0.0562 -0.1111 -0.0624 -0.1561 -0.0105 0.0271 -0.0011 0.0857 0.0516 -0.0387 -0.0856 0.0198 -1.1291 -0.0349 -0.0315 0.0705 -0.0057 -0.0195 -0.0522 0.0336 -0.0265 0.0823 0.0362 0.0892 -0.0831 -0.0747 0.1039 -0.0266 0.4814 0.0162 -0.0484 -0.0033 0.0761 -0.0312 0.0213 -0.0188 0.0121 -0.0537 0.0473 0.0583 0.0292 0.0655 0.0111 0.1129 0.0659 -0.0759 0.0795 -0.0425 -0.0335 -0.1145 0.01 0.0197 -0.1981 -0.0385 0.0319 0.0612 0.0273 -0.0384 0.035 -0.0085 -0.2808 -0.0483 0.1094 0.0

**La suite:**
1. Calcul du nouveau score grâce aux embeddings de mots des titres pour les 2900 couples
2. On tire au hasard 100 couples à partir des 2900 couples: 10 couples pour chaque décile
3. Annotation de ces 100 couples grâce à l'application Shiny
4. Avec le programme .R : nouvelle courbe des % de match bien prédits pour chaque décile
5. Avec le programme .R : nouvelle courbe de ROC basée sur le nouveau score

Calcul du cosine similarity pour ce couple de produits

In [0]:
# cf "read_glove_vecs" function MOOC Emoji = exactement le meme code 
def read_fasttext_vecs(vectors_file):
    with open(vectors_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            #print (np.array(line[1:], dtype=np.float64))
            #print (line[1:])
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [18]:
%%time
word_in_titles_bio_to_index, index_to_word_in_titles_bio, word_to_vec_map_fasttext_words_in_titles_bio = read_fasttext_vecs("data/cc.fr.300.vec_of_titles.txt")

CPU times: user 669 ms, sys: 43.3 ms, total: 712 ms
Wall time: 729 ms


In [0]:
def cosine_similarity(title_one,title_two, dict_of_embeddings):
    # calcul de l'embedding du titre 1
    sum_of_embeddings_one = 0
    for word in title_one.split(' '):
        #print (word)
        sum_of_embeddings_one += dict_of_embeddings[word]
    
    # On calcule en fait la moyenne des embeddings car on peut avoir un nombre de mots différent
    # entre les deux titres
    sum_of_embeddings_one = sum_of_embeddings_one / len(title_one.split(' '))
    
    #print (np.expand_dims(sum_of_embeddings_one, axis = 1).shape)
        
    # calcul de l'embedding du titre 2
    sum_of_embeddings_two = 0
    for word in title_two.split(' '):
        #print (word)
        sum_of_embeddings_two += dict_of_embeddings[word]
    #print (np.expand_dims(sum_of_embeddings_one, axis = 1).shape)
    
    # ajouter la moyenne
    sum_of_embeddings_two = sum_of_embeddings_two / len(title_two.split(' '))
    
    u = np.expand_dims(sum_of_embeddings_one, axis = 1) 
    v = np.expand_dims(sum_of_embeddings_two, axis = 1)
    resultat = np.sum(u*v) / (np.linalg.norm(u)*np.linalg.norm(v))
    #resultat = sklearn.metrics.pairwise.cosine_similarity(u,v)          
    return resultat

In [20]:
title_1 = "Baguette Classique Sans Gluten 360g Bio"
title_2 = "Cake Citron sans gluten 200gr"
cosine_similarity(title_1,title_2, word_to_vec_map_fasttext_words_in_titles_bio)

0.7174120672064965

In [21]:
title_1 = "Bio flan Citron 2x1/4l"
title_2 = "Bio Bioflan Citron sans sucre 7g"
cosine_similarity(title_1,title_2, word_to_vec_map_fasttext_words_in_titles_bio)

0.6732846897099303

In [22]:
title_1 = "Tamari Strong 145ml Bio"
title_2 = "Sauce Soja Tamari"
cosine_similarity(title_1,title_2, word_to_vec_map_fasttext_words_in_titles_bio)

0.6246775866959284

In [0]:
## 1. On charge les 2900 couples dans un dataframe

In [0]:
# df_all_couples = pd.read_excel("data/all_results_BIO_with_exhaust_and_titles_equal_20190529_124456_all.xlsx") 
df_all_couples = pd.read_pickle("data/all_results_final_all_with_filter.pkl")

In [0]:
# avec all_results_BIO_with_exhaust_and_titles_equal_20190529_124456_all.xlsx
# df_all_couples.shape

(2950, 17)

In [24]:
df_all_couples.shape

(2964, 17)

2. Calcul du nouveau score grâce aux embeddings de mots des titres

In [0]:
df_all_couples['xtitle_one_without_xa0'] = df_all_couples['xtitle_one'].map(lambda x:re.sub('\xa0',' ',x))
df_all_couples['xtitle_two_without_xa0'] = df_all_couples['xtitle_two'].map(lambda x:re.sub('\xa0',' ',x))

In [0]:
def compute_cosine_similarity(df):
    resultat = cosine_similarity(df["xtitle_one_without_xa0"],df["xtitle_two_without_xa0"], word_to_vec_map_fasttext_words_in_titles_bio)
    return resultat

In [0]:
df_all_couples["cosine_similarity_of_titles_fasttext"] = df_all_couples.apply(compute_cosine_similarity, axis = 1)

In [34]:
df_all_couples.sample(n=1)

,id_entities,couples,slug_one,slug_two,xtitle_one,xtitle_two,distributeur_one,distributeur_two,brand_one,brand_two,breadcrumb_one,breadcrumb_two,price_one,price_two,URL_one,URL_two,confidence_score,xtitle_one_without_xa0,xtitle_two_without_xa0,cosine_similarity_of_titles_fasttext
222,"(pate, dattes)","[[3398-pate-de-dattes-bio-6194000395113, Pâte de dattes, wbe, Rapunzel, Fruits secs, 3.06, pate dattes, 1.0], [rapunzel-pate-de-dattes-500g-p103139, Pâte de Dattes 500g, gwz, Rapunzel, Autres aides à la pâtisserie, 3.54, pate dattes 500g, 0.6666666666666666]]",rapunzel-pate-de-dattes-500g-p103139,3398-pate-de-dattes-bio-6194000395113,Pâte de Dattes 500g,Pâte de dattes,gwz,wbe,Rapunzel,Rapunzel,Autres aides à la pâtisserie,Fruits secs,3.54,3.06,https://www.greenweez.com/rapunzel-pate-de-dattes-500g-p103139,https://www.webecologie.com/fruits-secs/3398-pate-de-dattes-bio-6194000395113.html,0.833333,Pâte de Dattes 500g,Pâte de dattes,0.924663


In [35]:
df_all_couples[["xtitle_one","xtitle_two","cosine_similarity_of_titles_fasttext"]].head(n=20)

,xtitle_one,xtitle_two,cosine_similarity_of_titles_fasttext
0,Graines à Germer Bio RADIS NOIR 150g,Radis Noir Graines à Germer,0.820172
3,Gluten Pur,Gluten Pur,1.000000
0,Chips Lentilles Original 90g,Chips aux Lentilles Original 90g,0.884189
4,Sardines à l'huile d'olive,Sardines à l'Huile d'Olive,0.957535
1,Couscous Demi-complet 500g,Couscous Demi-complet 500g,1.000000
2,Bouillon cube Légumes aux Herbes,Bouillon de Légumes aux Herbes Cube,0.856759
1,Crème de Riz pour bébé,Crème de riz pour bébé,0.927083
2,Sésame Noir - 250g,Sésame Noir - 250g,1.000000
5,Café Grains BIO - Sélection 100% Arabica 1kg,Café Grains 100% Arabica,0.708431
16,Soupe Potimarron & courge Butternut 25cl,Soupe Potimarron & Courge Butternut,0.933894


Calcul de la nouvelle colonne strate pour chaque décile de "cosine_similarity_of_titles"

In [0]:
x_values = list(range(0,100,10))
perc_values = [df_all_couples[['cosine_similarity_of_titles_fasttext']].quantile(x/100)[0] for x in x_values]

In [0]:
def strate_percentiles(x):
    
    if x > 0 and x <= perc_values[0]:
        resultat = '1'
#         print (resultat)
    elif x > perc_values[0] and x <= perc_values[1]:
        resultat = '1'
#         print (resultat)
    elif x > perc_values[1] and x <= perc_values[2]:
        resultat = '2'
#         print (resultat)
    elif x > perc_values[2] and x <= perc_values[3]:
        resultat = '3'
#         print (resultat)
    elif x > perc_values[3] and x <= perc_values[4]:
        resultat = '4' 
#         print (resultat)
    elif x > perc_values[4] and x <= perc_values[5]:
        resultat = '5'
#         print (resultat)
    elif x > perc_values[5] and x <= perc_values[6]:
        resultat = '6'
#         print (resultat)
    elif x > perc_values[6] and x <= perc_values[7]:
        resultat = '7'
#         print (resultat)
    elif x > perc_values[7] and x <= perc_values[8]:
        resultat = '8'
#         print (resultat)
    elif x > perc_values[8] and x <= perc_values[9]:
        resultat = '9'
#         print (resultat)
#     elif x > perc_values[9] and x <= 1:
    else :
        resultat = '10'
#         print (resultat)

    return resultat

df_all_couples['strate_var_cosine_similarity_fasttext'] = df_all_couples['cosine_similarity_of_titles_fasttext'].map(strate_percentiles)

In [39]:
df_all_couples[df_all_couples["strate_var_cosine_similarity_fasttext"] == '1']

,id_entities,couples,slug_one,slug_two,xtitle_one,xtitle_two,distributeur_one,distributeur_two,brand_one,brand_two,breadcrumb_one,breadcrumb_two,price_one,price_two,URL_one,URL_two,confidence_score,xtitle_one_without_xa0,xtitle_two_without_xa0,cosine_similarity_of_titles_fasttext,strate_var_cosine_similarity_fasttext
75,"(puree, complete, d'amande)","[[4503-puree-d-amandes-completes-bio-350-g-3390390000115?PageSpeed=noscript, Purée d'Amande Complète, wbe, Jean Hervé, Pâte à tartiner bio, 8.89, puree d'amande complete, 1.0], [jean-herve-puree-d-amande-complete-5kg-p73162, Puree d'amande complète 5kg, gwz, Jean Hervé, Purées d'amandes, 114.03, puree d'amande complete 5kg, 0.75]]",jean-herve-puree-d-amande-complete-5kg-p73162,4503-puree-d-amandes-completes-bio-350-g-3390390000115?PageSpeed=noscript,Puree d'amande complète 5kg,Purée d'Amande Complète,gwz,wbe,Jean Hervé,Jean Hervé,Purées d'amandes,Pâte à tartiner bio,114.03,8.89,https://www.greenweez.com/jean-herve-puree-d-amande-complete-5kg-p73162,https://www.webecologie.com/pate-a-tartiner-bio/4503-puree-d-amandes-completes-bio-350-g-3390390000115.html?PageSpeed=noscript,0.875000,Puree d'amande complète 5kg,Purée d'Amande Complète,0.436755,1
76,"(puree, complete, d'amande)","[[4502-puree-d-amandes-completes-bio-700-g-3390390000122?PageSpeed=noscript, Purée d'Amande Complète, wbe, Jean Hervé, Pâte à tartiner bio, 17.43, puree d'amande complete, 1.0], [jean-herve-puree-d-amande-complete-5kg-p73162, Puree d'amande complète 5kg, gwz, Jean Hervé, Purées d'amandes, 114.03, puree d'amande complete 5kg, 0.75]]",jean-herve-puree-d-amande-complete-5kg-p73162,4502-puree-d-amandes-completes-bio-700-g-3390390000122?PageSpeed=noscript,Puree d'amande complète 5kg,Purée d'Amande Complète,gwz,wbe,Jean Hervé,Jean Hervé,Purées d'amandes,Pâte à tartiner bio,114.03,17.43,https://www.greenweez.com/jean-herve-puree-d-amande-complete-5kg-p73162,https://www.webecologie.com/pate-a-tartiner-bio/4502-puree-d-amandes-completes-bio-700-g-3390390000122.html?PageSpeed=noscript,0.875000,Puree d'amande complète 5kg,Purée d'Amande Complète,0.436755,1
121,"(chocolat noir, tartelettes)","[[bonneterre-tartelettes-chocolat-noir-bio-125g-p60368, Tartelettes chocolat noir bio 125g, gwz, Bonneterre, Autres biscuits, 3.19, tartelettes chocolat noir 125g, 0.75], [5955-tartelettes-au-chocolat-noir-3396413710092, Tartelettes au Chocolat Noir, wbe, Bonneterre, Biscuit bio, 3.30, tartelettes chocolat noir, 1.0]]",bonneterre-tartelettes-chocolat-noir-bio-125g-p60368,5955-tartelettes-au-chocolat-noir-3396413710092,Tartelettes chocolat noir bio 125g,Tartelettes au Chocolat Noir,gwz,wbe,Bonneterre,Bonneterre,Autres biscuits,Biscuit bio,3.19,3.30,https://www.greenweez.com/bonneterre-tartelettes-chocolat-noir-bio-125g-p60368,https://www.webecologie.com/biscuit-bio/5955-tartelettes-au-chocolat-noir-3396413710092.html,0.875000,Tartelettes chocolat noir bio 125g,Tartelettes au Chocolat Noir,0.438248,1
120,"(citron, safran, tortils)","[[2977-tortils-citron-safran-bio-3380390173506, Tortils Citron & Safran, wbe, Priméal, Pâtes bio, 3.15, tortils citron & safran, 0.75], [primeal-tortils-au-citron-et-au-safran-p5676, Tortils au Citron et au Safran, gwz, Priméal, Pâtes Parfumées, 2.82, tortils citron safran, 1.0]]",primeal-tortils-au-citron-et-au-safran-p5676,2977-tortils-citron-safran-bio-3380390173506,Tortils au Citron et au Safran,Tortils Citron & Safran,gwz,wbe,Priméal,Priméal,Pâtes Parfumées,Pâtes bio,2.82,3.15,https://www.greenweez.com/primeal-tortils-au-citron-et-au-safran-p5676,https://www.webecologie.com/pates-bio/2977-tortils-citron-safran-bio-3380390173506.html,0.875000,Tortils au Citron et au Safran,Tortils Citron & Safran,0.412378,1
161,sesame decortique,"[[markal-sesame-decortique-250g-p77434, Sésame décortiqué - 250g, gwz, Markal, Graines de Sésame, 2.04, sesame decortique 250g, 0.6666666666666666], [3142-sesame-decortique-bio-3329488521256, Sésame Décortiqué, wbe, Markal, Graines bio & Graines à germer, 2.05, sesame decorti

In [0]:
def return_sampled_df_by_strate(df_input, value_var): 
    df = df_input[df_input['strate_var_cosine_similarity_fasttext'] == value_var].sample(n=10)
    return df

In [0]:
all_sampled_100_couples_cos_sim = pd.concat([return_sampled_df_by_strate(df_all_couples,'1'),
                        return_sampled_df_by_strate(df_all_couples,'2'),
                        return_sampled_df_by_strate(df_all_couples,'3'),
                        return_sampled_df_by_strate(df_all_couples,'4'),
                        return_sampled_df_by_strate(df_all_couples,'5'),
                        return_sampled_df_by_strate(df_all_couples,'6'),
                        return_sampled_df_by_strate(df_all_couples,'7'),
                        return_sampled_df_by_strate(df_all_couples,'8'),
                        return_sampled_df_by_strate(df_all_couples,'9'),
                        return_sampled_df_by_strate(df_all_couples,'10')])

In [45]:
list(all_sampled_100_couples_cos_sim)

['id_entities',
 'couples',
 'slug_one',
 'slug_two',
 'xtitle_one',
 'xtitle_two',
 'distributeur_one',
 'distributeur_two',
 'brand_one',
 'brand_two',
 'breadcrumb_one',
 'breadcrumb_two',
 'price_one',
 'price_two',
 'URL_one',
 'URL_two',
 'confidence_score',
 'xtitle_one_without_xa0',
 'xtitle_two_without_xa0',
 'cosine_similarity_of_titles_fasttext',
 'strate_var_cosine_similarity_fasttext']

In [0]:
output_file_xlsx = "all_sampled_100_couples_cos_sim_fasttext.xlsx"
all_sampled_100_couples_cos_sim.to_excel(output_file_xlsx, index=False)

**On peut annoter ces 100 nouveaux couples en vue de voir si ce modèle est plus performant**

**Conclusion au sujet des différents modèles réalisables:**<br>
avec mesure de la performance du modèle pour 100 couples

On va avoir alors:<br>
- modèle n°1 fait cf Github
- modèle n°2 avec similarité calculée grâce aux word embeddings (FastText)
- modèle n°3 avec similarité calculée grâce aux entités embeddings (Word2vec + transfer learning)
- modèle n°4 avec similarité calculée grâce aux entités embeddings (Bert + transfer learning)
- modèle n°5 avec similarité calculée grâce aux encodings des images (architecture siamese network (à vérifier) + transfer learning)